In [1]:
from os import walk, getcwd, path

In [7]:
# This is our input: we are trying to find the office responsible
query_text = 'Sanierung von asbestbelasteten Gebäude'

In [6]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('aari1995/gBERT-large-sts-v2')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!git clone https://github.com/we-art-o-nauts/living-topics

Cloning into 'living-topics'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 45 (delta 2), reused 42 (delta 2), pack-reused 0
Unpacking objects: 100% (45/45), 49.83 KiB | 1.78 MiB/s, done.


In [41]:
# The passages are encoded as [ [title1, text1], [title2, text2], ...]
passages = []

# Get a list of objects at the root
froot = ['Bund', 'Gemeinde', 'Kantone', 'BK', 'EDA', 'EDI', 'EFD', 'EJPD', 'UVEK', 'VBS', 'WBF']
for dept in froot:
    print(dept)
    curdir = dept.lower()
    for (dirpath, dirnames, filenames) in walk(path.join(getcwd(), 'living-topics', curdir)):
        for filename in filenames:
            with open(path.join(dirpath, filename), mode="r", encoding="utf-8") as f:
                for line in f.readlines():
                    ll = line.strip()
                    if not ll:
                        continue
                    for llp in ll.split('.'):
                        llps = llp.strip()
                        if len(llps) > 20:
                            passages.append([ dept, llps ])


Bund
Gemeinde
Kantone
BK
EDA
EDI
EFD
EJPD
UVEK
VBS
WBF


In [42]:
# Calculate the vector space of the passages (SLOW)
passage_embedding = model.encode(passages)

In [54]:
# Encode our query for the model
query_embedding = model.encode(query_text)
print(query_text)

Sanierung von asbestbelasteten Gebäude


In [55]:
# Compute cosine-similarities to query
cosine_scores = util.cos_sim(query_embedding, passage_embedding)
print(len(cosine_scores[0]))

505


In [48]:
# Sort scores in decreasing order and show them
pairs = []
for i in range(len(cosine_scores[0])-1):
   pairs.append({'name': passages[i][0], 'text': passages[i][1], 'score': cosine_scores[0][i]})
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

In [63]:
for pair in pairs[0:3]:
    rounded = round(float(pair['score']), 3)
    print("{} \t Score: {} \t {}".format(pair['name'], rounded, pair['text']))

BK 	 Score: 0.603 	 Verwaltung von Unternehmensstammdaten und Zugang zu Gebäudeinformationen des Bundes werden vereinfacht
BK 	 Score: 0.602 	 Unter anderem soll eine neue Verordnung zur Datenbearbeitung erarbeitet werden
BK 	 Score: 0.597 	 2022 eine Sachverhaltsabklärung eröffnet


In [66]:
print("Sache der", pairs[0]['name'])

Sache der BK


# EOF